In [1]:
code = """première ligne
<a>
test dans a
<b class="prems deuz">
test dans b
</b>
<c> vide </c>
</a>
<d attribut='toto'> dans d </d>
"""
print(code)

première ligne
<a>
test dans a
<b class="prems deuz">
test dans b
</b>
<c> vide </c>
</a>
<d attribut='toto'> dans d </d>



# Structure d'arbre sur bs4

In [2]:
from bs4 import BeautifulSoup

In [3]:
soupe = BeautifulSoup(code, "lxml")

In [4]:
type(soupe)

bs4.BeautifulSoup

In [5]:
str(soupe)

'<html><body><p>première ligne\n<a>\ntest dans a\n<b class="prems deuz">\ntest dans b\n</b>\n<c> vide </c>\n</a>\n<d attribut="toto"> dans d </d>\n</p></body></html>'

In [6]:
type(soupe.contents)

list

In [7]:
len(soupe.contents)

1

In [8]:
enfant = soupe.contents[0]
type(enfant)

bs4.element.Tag

In [9]:
enfant.name

'html'

In [10]:
enfant.contents

[<body><p>première ligne
 <a>
 test dans a
 <b class="prems deuz">
 test dans b
 </b>
 <c> vide </c>
 </a>
 <d attribut="toto"> dans d </d>
 </p></body>]

In [11]:
interne = soupe.html.body.p
type(interne)

bs4.element.Tag

In [12]:
interne.name

'p'

In [13]:
print(interne.prettify())

<p>
 première ligne
 <a>
  test dans a
  <b class="prems deuz">
   test dans b
  </b>
  <c>
   vide
  </c>
 </a>
 <d attribut="toto">
  dans d
 </d>
</p>


In [14]:
len(interne.contents)

5

In [15]:
[type(c) for c in interne.contents]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [16]:
un, deux, trois, quatre, cinq = interne.contents

In [17]:
un

'première ligne\n'

In [18]:
type(un)

bs4.element.NavigableString

In [19]:
trois

'\n'

In [20]:
cinq

'\n'

In [21]:
deux

<a>
test dans a
<b class="prems deuz">
test dans b
</b>
<c> vide </c>
</a>

In [22]:
quatre

<d attribut="toto"> dans d </d>

In [23]:
quatre["attribut"]

'toto'

In [24]:
quatre.contents

[' dans d ']

In [25]:
quatre.get_text()

' dans d '

In [26]:
deux

<a>
test dans a
<b class="prems deuz">
test dans b
</b>
<c> vide </c>
</a>

In [27]:
deux.contents

['\ntest dans a\n',
 <b class="prems deuz">
 test dans b
 </b>,
 '\n',
 <c> vide </c>,
 '\n']

In [28]:
deux.parent

<p>première ligne
<a>
test dans a
<b class="prems deuz">
test dans b
</b>
<c> vide </c>
</a>
<d attribut="toto"> dans d </d>
</p>

# Requêtes bs4

In [29]:
print(soupe.prettify())

<html>
 <body>
  <p>
   première ligne
   <a>
    test dans a
    <b class="prems deuz">
     test dans b
    </b>
    <c>
     vide
    </c>
   </a>
   <d attribut="toto">
    dans d
   </d>
  </p>
 </body>
</html>


In [30]:
soupe.find("d")

<d attribut="toto"> dans d </d>

In [31]:
soupe.find_all(["c", "d"])

[<c> vide </c>, <d attribut="toto"> dans d </d>]

In [32]:
import re

In [33]:
soupe.find_all(re.compile("a|c"))

[<a>
 test dans a
 <b class="prems deuz">
 test dans b
 </b>
 <c> vide </c>
 </a>,
 <c> vide </c>]

In [34]:
def filtre(tag):
    return tag.name == "a" or (tag.has_attr("attribut") and tag["attribut"] == "toto")

In [35]:
soupe.find_all(filtre)

[<a>
 test dans a
 <b class="prems deuz">
 test dans b
 </b>
 <c> vide </c>
 </a>,
 <d attribut="toto"> dans d </d>]

In [36]:
soupe.find("b")

<b class="prems deuz">
test dans b
</b>

In [37]:
soupe.find("e")

In [38]:
soupe.b

<b class="prems deuz">
test dans b
</b>

In [39]:
soupe.b["class"]

['prems', 'deuz']

In [40]:
def requete(tag):
    return tag.has_attr("class") and tag["class"] == "prems deuz"

In [41]:
soupe.find(requete)

**Documentation officielle** de [beautiful soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

# Retour au CAC 40

**Exercice** Refaire la récupération de données sur la page du CAC40 via `bs4`, `re` et `requests`.

In [42]:
class ScrapingObsolete(Exception):
    pass

In [43]:
adresse = "https://fr.wikipedia.org/wiki/CAC_40"

In [44]:
from requests import get

In [45]:
requete = get(adresse)
if requete.status_code == 200:
    soupe = BeautifulSoup(requete.text)

In [46]:
def selection(tag):
    if tag.name != "table":
        return False
    resultat = tag.find("th")
    if resultat is None:
        return False
    return "Société" in resultat.get_text() 
    

In [47]:
resultat = soupe.find_all(selection)
if len(resultat) == 1:
    resultat, = resultat
else:
    raise ScrapingObsolete("Il y a plus d'un résultat")

In [48]:
[c.name for c in resultat.contents]

[None, 'tbody']

In [77]:
from dataclasses import dataclass
from typing import List, Tuple
from serde import serde
from serde.json import from_json, to_json

@serde
@dataclass
class Entreprise:
    nom: str
    lien: str
    secteur: str
    poids_indiciel: float
    chiffre_affaire_euros: int
    capitalisation_euros: int
    date_entree: Tuple[int, int, int]

**EXERCICE** Récupérer une liste d'objets de type `Entreprise` à partir de la table.

In [50]:
header, *lignes = resultat.find_all("tr")
if len(lignes) != 40:
    raise ScrapingObsolete("Il devrait y avoir 40 lignes pour 40 entreprises!")

In [51]:
def decoupe_ligne(ligne):
    return ligne.find_all("td")

In [52]:
societe, secteur, pi, ca, cb, di = decoupe_ligne(lignes[0])

In [53]:
def gere_societe(colonne) -> Tuple[str, str]:
    lien = colonne.find("a")
    return lien["title"], "https://fr.wikipedia.org" + lien["href"]

In [54]:
gere_societe(societe)

('Air liquide', 'https://fr.wikipedia.org/wiki/Air_liquide')

In [55]:
secteur

<td><a href="/wiki/Gaz_industriel" title="Gaz industriel">Gaz industriel</a>
</td>

In [56]:
def gere_secteur(colonne) -> str:
    lien = colonne.find("a")
    return lien["title"]

In [57]:
gere_secteur(secteur)

'Gaz industriel'

In [58]:
pi

<td>3,50
</td>

In [59]:
float("3,50")

ValueError: could not convert string to float: '3,50'

In [60]:
float("3.50")

3.5

In [61]:
def gere_pi(colonne) -> float:
    return float(colonne.get_text().strip().replace(",", "."))

In [62]:
gere_pi(pi)

3.5

In [63]:
ca

<td>20,49
</td>

In [64]:
def gere_ca(colonne) -> int:
    return int(10 ** 9 * float(colonne.get_text().strip().replace(",", ".")))

In [65]:
gere_ca(ca)

20490000000

In [66]:
cb

<td>66,56
</td>

In [67]:
def gere_cb(colonne) -> int:
    return int(10 ** 9 * float(colonne.get_text().strip().replace(",", ".")))

In [68]:
gere_cb(cb)

66560000000

In [69]:
di

<td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
</td>

In [70]:
def gere_di(colonne) -> Tuple[int, int, int]:
    chiffres = colonne.find("span")["data-sort-value"]
    return int(chiffres[0:4]), int(chiffres[4:6]), int(chiffres[6:8])

In [71]:
gere_di(di)

(1987, 12, 31)

In [78]:
resultat_final = list()
for ligne in lignes:
    societe, secteur, pi, ca, cb, di = decoupe_ligne(ligne)
    nom, lien = gere_societe(societe)
    resultat_final.append(
        Entreprise(
            nom = nom,
            lien = lien,
            secteur = gere_secteur(secteur),
            poids_indiciel = gere_pi(pi),
            chiffre_affaire_euros = gere_ca(ca),
            capitalisation_euros = gere_cb(cb),
            date_entree = gere_di(di),
        )
    )
    

In [79]:
resultat_final

[Entreprise(nom='Air liquide', lien='https://fr.wikipedia.org/wiki/Air_liquide', secteur='Gaz industriel', poids_indiciel=3.5, chiffre_affaire_euros=20490000000, capitalisation_euros=66560000000, date_entree=(1987, 12, 31)),
 Entreprise(nom='Airbus (groupe)', lien='https://fr.wikipedia.org/wiki/Airbus_(groupe)', secteur='Aéronautique', poids_indiciel=3.87, chiffre_affaire_euros=49910000000, capitalisation_euros=74870000000, date_entree=(1990, 2, 5)),
 Entreprise(nom='Alstom', lien='https://fr.wikipedia.org/wiki/Alstom', secteur='Constructeur ferroviaire', poids_indiciel=0.43, chiffre_affaire_euros=8780000000, capitalisation_euros=8170000000, date_entree=(2020, 9, 21)),
 Entreprise(nom='ArcelorMittal', lien='https://fr.wikipedia.org/wiki/ArcelorMittal', secteur='Acier', poids_indiciel=1.16, chiffre_affaire_euros=47570000000, capitalisation_euros=18980000000, date_entree=(2003, 11, 21)),
 Entreprise(nom='Axa', lien='https://fr.wikipedia.org/wiki/Axa', secteur='Assurance', poids_indiciel=

**EXERCICE** Utiliser `pyserde` et modifier la dataclass `Entreprise` pour sérialiser vers `json` et `yaml` le résultat final dans des fichiers `stockage.json` et `stockage.yaml`.

In [80]:
code_json = to_json(resultat_final)

In [81]:
with open("stockage.json", "w") as fichier:
    fichier.write(code_json)

In [82]:
with open("stockage.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [83]:
print(contenu_fichier)

[{"nom":"Air liquide","lien":"https://fr.wikipedia.org/wiki/Air_liquide","secteur":"Gaz industriel","poids_indiciel":3.5,"chiffre_affaire_euros":20490000000,"capitalisation_euros":66560000000,"date_entree":[1987,12,31]},{"nom":"Airbus (groupe)","lien":"https://fr.wikipedia.org/wiki/Airbus_(groupe)","secteur":"Aéronautique","poids_indiciel":3.87,"chiffre_affaire_euros":49910000000,"capitalisation_euros":74870000000,"date_entree":[1990,2,5]},{"nom":"Alstom","lien":"https://fr.wikipedia.org/wiki/Alstom","secteur":"Constructeur ferroviaire","poids_indiciel":0.43,"chiffre_affaire_euros":8780000000,"capitalisation_euros":8170000000,"date_entree":[2020,9,21]},{"nom":"ArcelorMittal","lien":"https://fr.wikipedia.org/wiki/ArcelorMittal","secteur":"Acier","poids_indiciel":1.16,"chiffre_affaire_euros":47570000000,"capitalisation_euros":18980000000,"date_entree":[2003,11,21]},{"nom":"Axa","lien":"https://fr.wikipedia.org/wiki/Axa","secteur":"Assurance","poids_indiciel":2.69,"chiffre_affaire_euros":

In [84]:
type(contenu_fichier)

str

In [85]:
reconstitution = from_json(List[Entreprise], contenu_fichier)

In [86]:
type(reconstitution)

list

In [87]:
reconstitution[0]

Entreprise(nom='Air liquide', lien='https://fr.wikipedia.org/wiki/Air_liquide', secteur='Gaz industriel', poids_indiciel=3.5, chiffre_affaire_euros=20490000000, capitalisation_euros=66560000000, date_entree=(1987, 12, 31))

# Problème avec les requêtes http pures

In [74]:
req = get("https://www.leboncoin.fr/recherche?text=appartement&locations=Tours_37000__47.38846_0.68957_5000")

In [75]:
req.status_code

403

In [76]:
req.text

'<html><head><title>leboncoin.fr</title><meta property="og:title" content="Rendez-vous sur leboncoin pour découvrir cette annonce !" />\r\n<meta property="og:image" content="https://img.datadome.co/captcha/page-customization/1872/866d27bc-26b6-476e-b41d-496f3e0a7fb4.jpeg" /><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={\'cid\':\'AHrlqAAAAAMAhKGTnx-7bE8AVsQ_mw==\',\'hsh\':\'05B30BD9055986BD2EE8F5A199D973\',\'t\':\'bv\',\'s\':2089,\'e\':\'c34e3101506f7addcc479eaf45c6cc301f66f33a8a23b7549fbfb8fa5a8fbd1d\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>\n'